In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import sys
import random
import os
import matplotlib.image as mpimg
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [34]:
writer = SummaryWriter(f'runs/rough/')

In [35]:
plt.figure(figsize=(20,20))
IMG_WIDTH=64
IMG_HEIGHT=64

<Figure size 1440x1440 with 0 Axes>

In [36]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            d = cv2.imread(image_path)
            image= cv2.cvtColor( d, cv2.COLOR_BGR2GRAY)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
img_data, class_name = create_dataset('archive/seg_train/seg_train')

In [37]:
class imgdataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.n_samples = x.shape[0]
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.n_samples

In [38]:
img_data = np.asarray(img_data,dtype=np.float32)
img_data = torch.FloatTensor(img_data)
img_data = img_data.reshape(-1,IMG_WIDTH*IMG_HEIGHT)
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]
target_val = np.array(target_val,dtype=np.longlong)
target_val = torch.LongTensor(target_val)
print(img_data.shape, target_val.shape)

torch.Size([14034, 4096]) torch.Size([14034])


In [56]:
n_features = img_data.shape[1]
batch_size = 64
learning_rate = 0.01
n_classes = 6
n_epoch = 5

In [57]:
data = imgdataset(img_data,target_val)
X_dataloader = DataLoader(dataset=data,batch_size=batch_size,shuffle=True)

In [58]:
examples = iter(X_dataloader)
p,q = examples.next()
print(p.shape,q.shape)
print(p[49])

torch.Size([64, 4096]) torch.Size([64])
tensor([0.9961, 0.9961, 0.9961,  ..., 0.1686, 0.2039, 0.2196])


In [59]:
class nnmodel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(nnmodel, self).__init__()
        self.r = nn.ReLU()
        self.l1 = nn.Linear(input_dim,128)
        self.l2 = nn.Linear(128,64)
        self.l3 = nn.Linear(64,32)
        self.l4 = nn.Linear(32,16)
        self.l5 = nn.Linear(16,8)
        self.pred = nn.Linear(8,output_dim)
    def forward(self,x):
        out = self.l1(x)
        out = self.r(out)
        out = self.l2(out)
        out = self.r(out)
        out = self.l3(out)
        out = self.r(out)
        out = self.l4(out)
        out = self.r(out)
        out = self.l5(out)
        out = self.r(out)
        out = self.r(self.pred(out))
        return out

In [60]:
class nnmodel1(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(nnmodel1, self).__init__()
        self.l1 = nn.Linear(input_dim,100)
        self.t1 = nn.ReLU()
        self.l2 = nn.Linear(100,100)
        self.t2 = nn.ReLU()
        self.l3 = nn.Linear(100,100)
        self.t3 = nn.ReLU()
        self.l4 = nn.Linear(100,100)
        self.t4 = nn.ReLU()
        self.l5 = nn.Linear(100,100)
        self.t5 = nn.ReLU()
        self.pred = nn.Linear(100,output_dim)
    def forward(self,x):
        out = self.l1(x)
        out = self.t1(out)
        out = self.l2(out)
        out = self.t2(out)
        out = self.l3(out)
        out = self.t3(out)
        out = self.l4(out)
        out = self.t4(out)
        out = self.l5(out)
        out = self.t5(out)
        out = self.pred(out)
        return out

In [61]:
class nnmodel2(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(nnmodel2, self).__init__()
        self.l1 = nn.Linear(input_dim,64*64)
        #self.l1.weight = torch.nn.init.xavier_normal_(self.l1.weight,1.0)
        #print(self.l1.weight)
        self.t1 = nn.LeakyReLU()
        self.l5 = nn.Linear(64*64,32*32)
        #self.l5.weight = torch.nn.init.kaiming_normal_(self.l5.weight)
        #print(self.l5.weight)
        self.t5 = nn.LeakyReLU()
        self.l2 = nn.Linear(32*32,16*16)
        #self.l2.weight = torch.nn.init.xavier_uniform_(self.l2.weight)
        #print(self.l2.weight)
        self.t2 = nn.LeakyReLU()
        self.l3 = nn.Linear(16*16,8*8)
        #self.l3.weight = torch.nn.init.kaiming_uniform_(self.l3.weight)
        self.t3 = nn.LeakyReLU()
        self.l4 = nn.Linear(8*8,4*4)
        #self.l4.weight = torch.nn.init.xavier_normal_(self.l4.weight)
        self.t4 = nn.LeakyReLU()
        self.pred = nn.Linear(4*4,output_dim)
        #self.pred.weight = torch.nn.init.kaiming_normal_(self.pred.weight)
    def forward(self,x):
        out = self.l1(x)
        out = self.t1(out)
        out = self.l5(out)
        out = self.t5(out)
        out = self.l2(out)
        out = self.t2(out)
        out = self.l3(out)
        out = self.t3(out)
        out = self.l4(out)
        out = self.t4(out)
        out = self.pred(out)
        return out

In [62]:
model = nnmodel2(n_features,n_classes)

Parameter containing:
tensor([[-0.0055, -0.0072, -0.0047,  ...,  0.0072,  0.0100,  0.0126],
        [ 0.0151,  0.0015,  0.0099,  ..., -0.0047, -0.0049, -0.0137],
        [-0.0101, -0.0101, -0.0152,  ...,  0.0004, -0.0021, -0.0114],
        ...,
        [ 0.0129,  0.0092, -0.0026,  ..., -0.0106, -0.0093, -0.0137],
        [-0.0025, -0.0121,  0.0113,  ...,  0.0025,  0.0034,  0.0006],
        [ 0.0126, -0.0130, -0.0065,  ...,  0.0080,  0.0057, -0.0022]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0135, -0.0140, -0.0009,  ...,  0.0107,  0.0152,  0.0015],
        [ 0.0056,  0.0083,  0.0144,  ...,  0.0113,  0.0142,  0.0084],
        [-0.0113, -0.0021, -0.0132,  ...,  0.0043,  0.0151, -0.0115],
        ...,
        [-0.0056, -0.0134, -0.0057,  ...,  0.0111,  0.0018, -0.0089],
        [-0.0076, -0.0102,  0.0088,  ..., -0.0155, -0.0074, -0.0019],
        [-0.0029, -0.0040,  0.0097,  ..., -0.0011, -0.0012, -0.0087]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.

In [63]:
writer.add_graph(model, p[2])

In [64]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [65]:
n_total_epochs = len(X_dataloader)
for epoch in range(n_epoch):
    for i, (images,labels) in enumerate(X_dataloader):
        outputs = model(images)
        loss = loss_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 50 ==0 :
            print(f'epochs :{epoch+1}/{n_epoch}, step: {i+1}/{n_total_epochs}, loss : {loss}')

epochs :1/5, step: 50/220, loss : 1.824745774269104
epochs :1/5, step: 100/220, loss : 1.735074520111084
epochs :1/5, step: 150/220, loss : 1.6869785785675049
epochs :1/5, step: 200/220, loss : 1.52239191532135
epochs :2/5, step: 50/220, loss : 1.7479628324508667
epochs :2/5, step: 100/220, loss : 1.608888864517212
epochs :2/5, step: 150/220, loss : 1.658441185951233
epochs :2/5, step: 200/220, loss : 1.6477131843566895
epochs :3/5, step: 50/220, loss : 1.6866381168365479
epochs :3/5, step: 100/220, loss : 1.4931752681732178
epochs :3/5, step: 150/220, loss : 1.6046814918518066
epochs :3/5, step: 200/220, loss : 1.5873149633407593
epochs :4/5, step: 50/220, loss : 1204.036376953125
epochs :4/5, step: 100/220, loss : 16264.9130859375
epochs :4/5, step: 150/220, loss : 15624.27734375
epochs :4/5, step: 200/220, loss : 3326.083251953125
epochs :5/5, step: 50/220, loss : 195.35067749023438
epochs :5/5, step: 100/220, loss : 134.62899780273438
epochs :5/5, step: 150/220, loss : 34.630348205

In [66]:
test_img, test_classes = create_dataset('archive/seg_test/seg_test')

In [67]:
print(len(test_img))
test_img = np.asarray(test_img)
test_img = torch.FloatTensor(test_img)
test_img = test_img.reshape(-1,IMG_WIDTH*IMG_HEIGHT)

3000


torch.Size([3000, 4096])

In [98]:
print(test_img.shape)

torch.Size([3000, 4096])


In [68]:
test_labels_dict = {k: v for v,k in enumerate(np.unique(test_classes))}


In [69]:
test_labels = [test_labels_dict[class_name[i]] for i in range(len(test_classes))]


In [70]:
test_labels = np.asarray(test_labels,dtype=np.longlong)
test_labels = torch.LongTensor(test_labels)
test_labels.shape

torch.Size([3000])

In [71]:
test_data = imgdataset(test_img,test_labels)

In [99]:
test_loader = DataLoader(dataset=test_loader,batch_size=batch_size,shuffle=True)

In [100]:
n_total_epochs_test = len(test_loader)
for epoch in range(n_epoch):
    for i, (images,labels) in enumerate(test_loader):
        outputs = model(images)
        loss1 = loss_criterion(outputs, labels)
        if (i+1) % 1000 ==0 :
            print(f'epochs :{epoch+1}/{n_epoch}, step: {i+1}/{n_total_epochs_test}, loss : {loss1}')

In [107]:
m = nn.Softmax(dim=0)
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images, labels in test_loader:
        outputs = model(images)
        outputs = m(outputs)
        _, predictions = torch.max(outputs, 1)
        n_samples+=labels.shape[0]
        n_correct = (predictions == labels).sum().item()
    acc = 100.0 * n_correct/n_samples
    print(f'accuracy = {acc}')

accuracy = 0.7666666666666667
